# Module 5 Homework
In this homework we'll put what we learned about Spark in practice.

In [1]:
# Install PySpark and ngrok
!pip install pyspark[sql,pandas_on_spark,connect]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 55.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.29.3-cp38-abi3-manylinux2014_x86_64.whl (319 kB)
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747859 sha256=8c144292faf4523cc50b4505b6fef80c61518c8cebc2fa354697bb78b768e061
  Stored in directory: /home/majeedk/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have pro

In [2]:
# Import necessary libraries
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [5]:
# Start spark session
spark = SparkSession.builder.appName("LocalSparkUI").getOrCreate()

## Question 1: Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

> [!NOTE]
> To install PySpark follow this [guide](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/05-batch/setup/pyspark.md)

In [6]:
# --- Question 1: Install Spark and PySpark ---
print("Spark Version:", spark.version)

Spark Version: 3.5.5


In [7]:
# --- Download Data ---
!mkdir -p data  # Create the 'data' directory if it doesn't exist
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
!wget -P data https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-06 18:47:25--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.133, 18.239.238.212, 18.239.238.152, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘data/yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  62.5MB/s    in 1.0s    

2025-03-06 18:47:26 (62.5 MB/s) - ‘data/yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]

--2025-03-06 18:47:27--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.152, 18.239.238.119, 18.239.238.212, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.152|:443... connected.
HTTP req

## Question 2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [8]:
df = spark.read.parquet("data/yellow_tripdata_2024-10.parquet")
df_repartitioned = df.repartition(4)
df_repartitioned.write.parquet("data/yellow_tripdata_2024-10-repartitioned.parquet")

In [16]:
 # Calculate average file size
parquet_files = [f for f in os.listdir("data/yellow_tripdata_2024-10-repartitioned.parquet") if f.endswith(".parquet")]
total_size = 0
for file in parquet_files:
    total_size += os.path.getsize(os.path.join("data/yellow_tripdata_2024-10-repartitioned.parquet", file))
average_size_mb = (total_size / len(parquet_files)) / (1024 * 1024)
print(f"Average Parquet file size: {average_size_mb} MB")

Average Parquet file size: 22.394158363342285 MB


In [17]:
from pyspark.sql.functions import input_file_name, sum

# Read Parquet files using binaryFiles
file_sizes = spark.read.format("binaryFile").load("data/yellow_tripdata_2024-10-repartitioned.parquet/*.parquet") \
    .select(input_file_name().alias("path"), "length")  # Select 'length' column directly

# Calculate the total size and count of files
total_size = file_sizes.agg(sum("length")).collect()[0][0]
num_files = file_sizes.count()

# Calculate the average size in MB
average_size_mb = (total_size / num_files) / (1024 * 1024)

print(f"Average Parquet file size: {average_size_mb} MB")

Average Parquet file size: 22.394158363342285 MB


## Question 3: Count records 

How many taxi trips were there on the 15th of October?

Consider only trips that started on the 15th of October.

In [18]:
# Using Spark SQL
# Register the DataFrame as a temporary view
df.createOrReplaceTempView("trip_data")

In [19]:
# Use Spark SQL to count trips on October 15th
count = spark.sql(
    """
    SELECT COUNT(*) AS trip_count
    FROM trip_data
    WHERE DATE(tpep_pickup_datetime) = '2024-10-15'
    """
).collect()[0][0]

print(f"Taxi trips on October 15th: {count}")

Taxi trips on October 15th: 128893


In [20]:
# Using data frame instead of SQL
count = df.filter(F.to_date(df.tpep_pickup_datetime) == '2024-10-15').count()
print(f"Taxi trips on October 15th: {count}")

Taxi trips on October 15th: 128893


## Question 4: Longest trip

What is the length of the longest trip in the dataset in hours?

In [21]:
# Use Spark SQL to calculate the longest trip in hours
longest_trip_hours = spark.sql(
    """
    SELECT MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600) AS longest_trip_hours
    FROM trip_data
    """
).collect()[0][0]

print(f"Longest trip in hours: {longest_trip_hours}")

Longest trip in hours: 162.61777777777777


In [22]:
# Using Data Frame instead
longest_trip_hours = df.agg(
    F.max(
        (F.unix_timestamp(F.col("tpep_dropoff_datetime")) - F.unix_timestamp(F.col("tpep_pickup_datetime"))) / 3600
    )
).collect()[0][0]
print(f"Longest trip in hours: {longest_trip_hours}")

Longest trip in hours: 162.61777777777777


## Question 5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

![Image](img/SparkUI_Local.png)

## Question 6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [25]:
# Using SQL
# Load zone lookup data into a temporary view
zone_df = spark.read.csv("data/taxi_zone_lookup.csv", header=True)
zone_df.createOrReplaceTempView("zone_lookup")

# Use Spark SQL to find the least frequent pickup zone
least_frequent_zone = spark.sql(
    """
    SELECT Zone, COUNT(*) AS pickup_count
    FROM trip_data
    JOIN zone_lookup ON trip_data.PULocationID = zone_lookup.LocationID
    GROUP BY Zone
    ORDER BY pickup_count ASC
    LIMIT 1
    """
).collect()[0][0]  # Extract the Zone value

print(f"Least frequent pickup zone: {least_frequent_zone}")

Least frequent pickup zone: Governor's Island/Ellis Island/Liberty Island


In [27]:
df_with_zones = df.join(zone_df, df.PULocationID == zone_df.LocationID, "left")

pickup_counts = df_with_zones.groupBy("Zone").count()
least_frequent_zone = pickup_counts.orderBy("count").first().Zone
print(f"Least frequent pickup zone: {least_frequent_zone}")

Least frequent pickup zone: Governor's Island/Ellis Island/Liberty Island


In [28]:
spark.stop()